# Imports

In [ ]:
import face_recognition as face_file
import pandas as pd
import numpy as np
import cv2

In [ ]:
face_file.r.ping() # test if system is connected to redis cloud or not

# Extract Data from Redis to Compare

**Key-name: 'academy:register'**

In [ ]:
key_name = 'academy:register'
redis_hash = face_file.r.hgetall(key_name)
print(redis_hash)
redis_series = pd.Series(redis_hash)
redis_series


In [ ]:
# Convert bytes to array
redis_series = redis_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))
index = redis_series.index
index = list(map(lambda x: x.decode(), index)) # converting keys to strings

In [ ]:
redis_series.index = index

In [ ]:
redis_df = redis_series.to_frame().reset_index() # converting to df and resetting index
redis_df.columns = ['name_role', 'facial_features']
redis_df[['Name' , 'Role']] = redis_df['name_role'].apply(lambda x: x.split('@')).apply(pd.Series) # splitting name and role to different columns
redis_df

# Real time Predictions

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if ret == False:
        break
    pred_frame = face_file.face_prediction(image=frame, df=redis_df)
    
    cv2.imshow('Frame', frame)
    cv2.imshow('Pred Frame', pred_frame)
    
    if cv2.waitKey(1) == 27:
        break
cv2.destroyAllWindows()
cap.release()